<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap12_generative_model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q accelerate==0.31.0 peft==0.11.1 bitsandbytes==0.43.1 transformers==4.41.2 trl==0.9.4 sentencepiece==0.2.0 triton==3.1.0 datasets==2.20.0

12.3 QLoRA による指示チューニング

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

# チャットテンプレートを使用するためのトークナイザーを読み込む
template_tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlma-1.1BChat-v1.0"
)

def format_prompt(example):
  """TinyLLmaが使用している<|user|>テンプレートを用いてプロンプトをフォーマットする"""

  # 回答をフォーマットする
  chat = example["messages"]
  prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

  return {"text": prompt}

# データセットを読み込み、フォーマットする
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
    .shuffle(seed=42)
    .select(range(3_000))
)
dataset = dataset.map(format_prompt)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: TinyLlama/TinyLlma-1.1BChat-v1.0 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# フォーマットされたテンプレートの例
print(dataset["text"][2576])

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlma/TinyLlma-1.1B-intermediate-step-1431k-3T"

# 4ビット量子化設定(QLoRA の Q)
bnb_config = BitsAndBytesConfig(
    laod_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# GPU で学習するためにモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
 )
model.config.use_cashe = False
model.config.pretraining_tp = 1

# トークナイザーを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"